# MPNN for edge betweenness

The notebook will train a model to compute edge betweenness of edges of a graph, in an inductive setting allowing for generalization to unseen graphs.
This model will be used to predict edge betweenness inside the Girvan-Newman algorithm by networkx library.
Steps:
1. get a dataset to detect community in
2. train an edge classifier (not node) for edge betweenness
3. save this model to disk
4. load this model and predict edge betweenness on unseen graphs
5. use networkx Girvan-Newman algorithm with a replacement for best edge selection routine, a replacmenet that loads the model and predict edge betweenness

# 1 Dataset

## 1.1 datasets available:
multiple graphs in wihch we can compute edge betweenness
Choice: MUTAG, ENZYMES, PROTEINS...

## 1.2 target:
we need the edge betweenness of each edge,  which we can compute with networkx for example

## 1.3 dataset creation
1. get the graphs from MUTAG
2. transform to networkx
3. compute the edge betweenness
4. save to networkx graph format
5. transform to PyTorch_Geometric dataset
6. pass this dataset to model training task

# 2 Model 

## 2.1 Choice of the graph neural network
Choose or create a GNN that 
1. considers and __predicts__ edge features!
2. works in an inductive setting (predicts on unseen graphs)

Ideas:
- Link prediction algorithm
- Link prediction algorithm modified?
- MPNN
- GraphSAGE modified?
- Graph Network (DeepMind)

models allowing edge features:
- GNN
- MPNN
- DCNN
- PATCHY-SAN
- DGCNN [Deep Graph Convolutional Neural Network](https://www.groundai.com/project/link-prediction-based-on-graph-neural-networks/)
- EGNNA

model modif -> predict values edges
    betweenness associated to shortedst path -> features nodes not needed for example
maybe node features are also useful...
additional node in each edge
transform nodes to edges
min betweens in both vertex=edge betweenness
   

## 2.2 Implementation
Available implementation or build. PyTorchGeometric seems a good tool to implement models..


## 2.3 Hyperparameter search
Which hyperparams to search for optimal performance?

# 3 Modify Girvan-Newman

Modify the Girvan-Newman algorithm by calling a new function that:
    1. loads the GNN model from disk
    2. predicts the edge betweenness of all edges of a graph
    3. ranks the edges according to that edge betweenness
The rest is already taken car by networkx.

Time the improvement: time original and new Girvan-Newman execution and see difference in times

In [1]:
# modifying edge_betweenness
from networkx import edge_betweenness_centrality as betweenness


def predict_all_betweennesses(G):
    centrality = {}
    
    # transform networkx graph to PyTorch Geometric Dataset
    
    
    # load model from disk
    
    
    # predict all betweennesses of edges of G
    
    # save into centrality dict 
    # { (n1,n2): 0.0938, (n3,n4): 0.1230, ..}
    return centrality
    

def most_central_edge(G):
    centrality = betweenness(G)
    return max(centrality.values())

G = nx.read_gml('football.gml')
nx.set_edge_attributes(G, {(u,v): 1 for u, v in G.edges()},'weight')
comp = girvan_newman(G, most_valuable_edge=most_central_edge)
tuple(sorted(c) for c in next(comp))

NameError: name 'nx' is not defined